# all text encoders

In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)

In [2]:
import json

with open('../pdf_to_dictionary/english_syn_ant.json', encoding='utf-8') as f:
    d = json.load(f)

In [3]:
import numpy as np
from numpy.linalg import norm

def my_cosine_similarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

In [4]:
# make smaller test dict
d_keys_li=list(d.keys())
smaller_dict={}
for i in range(0,10):
    smaller_dict[d_keys_li[i]]=d[d_keys_li[i]]
# pp.pprint(smaller_dict)

In [6]:
# d - full test disctionary
# smaller_dict - 10 key dictionary
DICT = smaller_dict
#DICT = d

In [7]:
for baseword,v in DICT.items():
    print(baseword,v["syn"],v["ant"])
    break

aback ['backwards', 'rearwards', 'aft', 'abaft', 'astern', 'behind', 'back'] ['onwards', 'forwards', 'ahead', 'before', 'afront', 'beyond', 'afore']


In [ ]:
def get_avg_res_bert(tokenizer,model):
    from scipy.spatial import distance
    import numpy as np

    sum=0
    cnt=0
    avg=0

    for k,v in DICT.items():
        baseword=k
        encoded_input_base=tokenizer(baseword,return_tensors='pt')
        output_base = model(**encoded_input_base)
        for e in v[1]:
            synonym=e
            encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
            output_synonym = model(**encoded_input_synonym)

            embedded_base_avg=np.average(output_base.last_hidden_state[0].detach().numpy(), axis=0)
            embedded_synonym_avg=np.average(output_synonym.last_hidden_state[0].detach().numpy(), axis=0)
            dist=distance.cosine(embedded_base_avg,embedded_synonym_avg)
            sum=sum+dist
            cnt=cnt+1
    # overall result on synonym-dictionary
        print(cnt, end="\r")
           
    avg=sum/cnt
    print(avg)

In [ ]:
def get_avg_res_bert(tokenizer, model):
    from scipy.spatial import distance
    import numpy as np
    sum_syn=0
    sum_ant=0
    cnt_syn=0
    cnt_ant=0

    sum=0
    cnt=0
    avg=0

    for baseword,v in DICT.items():
        encoded_input_base=tokenizer(baseword,return_tensors='pt')
        output_base = model(**encoded_input_base,output_hidden_states=True)
        embedded_base_avg=np.average(output_base.last_hidden_state[0].detach().numpy(), axis=0)

        for synonym in v["syn"]:
            encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
            output_synonym = model(**encoded_input_synonym,output_hidden_states=True)
            embedded_synonym_avg=np.average(output_synonym.last_hidden_state[0].detach().numpy(), axis=0)
            sim_syn=my_cosine_similarity(embedded_base_avg,embedded_synonym_avg)
            sum_syn=sum_syn+sim_syn
            cnt_syn=cnt_syn+1
        
        for antonym in v["ant"]:
            encoded_input_antonym=tokenizer(antonym,return_tensors='pt')
            output_antonym = model(**encoded_input_antonym,output_hidden_states=True)
            embedded_antonym_avg=np.average(output_antonym.last_hidden_state[0].detach().numpy(), axis=0)
            sim_ant=my_cosine_similarity(embedded_base_avg,embedded_antonym_avg)
            sum_ant=sum_ant+sim_ant
            cnt_ant=cnt_ant+1
            
        print(cnt_syn, cnt_ant, end="\r")
           
    print("syn", sum_syn/cnt_syn)
    print("ant", sum_ant/cnt_ant)

In [ ]:
def get_avg_res_other(tokenizer, model):
    from scipy.spatial import distance
    import numpy as np
    sum_syn=0
    sum_ant=0
    cnt_syn=0
    cnt_ant=0

    sum=0
    cnt=0
    avg=0

    for baseword,v in DICT.items():
        encoded_input_base=tokenizer(baseword,return_tensors='pt')
        output_base = model(**encoded_input_base,output_hidden_states=True)
        output_base_np=output_base.hidden_states[-1][0].detach().numpy()
        embedded_base_avg=np.average(output_base_np, axis=0)

        for synonym in v["syn"]:
            encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
            output_synonym = model(**encoded_input_synonym,output_hidden_states=True)
            output_synonym_np=output_synonym.hidden_states[-1][0].detach().numpy()
            embedded_synonym_avg=np.average(output_synonym_np, axis=0)
            sim_syn=my_cosine_similarity(embedded_base_avg,embedded_synonym_avg)
            sum_syn=sum_syn+sim_syn
            cnt_syn=cnt_syn+1
        
        for antonym in v["ant"]:
            encoded_input_antonym=tokenizer(antonym,return_tensors='pt')
            output_antonym = model(**encoded_input_antonym,output_hidden_states=True)
            output_antonym_np=output_antonym.hidden_states[-1][0].detach().numpy()
            embedded_antonym_avg=np.average(output_antonym_np, axis=0)
            sim_ant=my_cosine_similarity(embedded_base_avg,embedded_antonym_avg)
            sum_ant=sum_ant+sim_ant
            cnt_ant=cnt_ant+1
            
        print(cnt_syn, cnt_ant, end="\r")
           
    print("syn", sum_syn/cnt_syn)
    print("ant", sum_ant/cnt_ant)

In [ ]:
# bert-base-multilingual-uncased
# Run time: 21m
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

get_avg_res_bert(tokenizer,model)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.6882242571030345


In [ ]:
# bert-base-multilingual-cased
# Run time: 19m
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")

get_avg_res_bert(tokenizer,model)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.5303627210003989


In [ ]:
# hubert-base-cc
# Run time: 18m
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("SZTAKI-HLT/hubert-base-cc")
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")

get_avg_res_bert(tokenizer,model)

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.9372422460998807


In [ ]:
# text-generation-news-gpt2-small-hungarian
# Run time: 21m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NYTK/text-generation-news-gpt2-small-hungarian")
model = AutoModelForCausalLM.from_pretrained("NYTK/text-generation-news-gpt2-small-hungarian")

get_avg_res_other(tokenizer,model)

0.5411062889865467


In [ ]:
# text-generation-poem-petofi-gpt2-small-hungarian
# Run time: 21m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NYTK/text-generation-poem-petofi-gpt2-small-hungarian")
model = AutoModelForCausalLM.from_pretrained("NYTK/text-generation-poem-petofi-gpt2-small-hungarian")

get_avg_res_other(tokenizer,model)

0.5889360691819873


In [ ]:
# mGPT
# Run time: 234m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/mGPT")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/mGPT")

get_avg_res_other(tokenizer,model)

0.850671974675996


In [ ]:
# sentiment-hts2-xlm-roberta-hungarian
# Run time: 11m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts2-xlm-roberta-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts2-xlm-roberta-hungarian")

get_avg_res_other(tokenizer,model)

0.24903443138880124


In [ ]:
# sentiment-hts5-xlm-roberta-hungarian
# Run time: 18m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts5-xlm-roberta-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts5-xlm-roberta-hungarian")

get_avg_res_other(tokenizer,model)

0.7542195948106902


In [ ]:
# sentiment-hts5-hubert-hungarian
# Run time: 18m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts5-hubert-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts5-hubert-hungarian")

get_avg_res_other(tokenizer,model)

0.8388932666608265


In [ ]:
# sentiment-hts2-hubert-hungarian
# Run time: 17m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts2-hubert-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts2-hubert-hungarian")

get_avg_res_other(tokenizer,model)

0.7557218223810196


In [ ]:
# xlm-roberta-base
# Run time: 43m
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

get_avg_res_other(tokenizer,model)

0.9931744592530387


In [ ]:
# xlm-roberta-large
# Run time: 82m
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

get_avg_res_other(tokenizer,model)

0.9933422335556575


In [ ]:
# xlm-mlm-100-1280
# Run time: 109m
from transformers import XLMTokenizer, XLMWithLMHeadModel
import torch

tokenizer = XLMTokenizer.from_pretrained("xlm-mlm-100-1280")
model = XLMWithLMHeadModel.from_pretrained("xlm-mlm-100-1280")

get_avg_res_other(tokenizer,model)

Some weights of XLMWithLMHeadModel were not initialized from the model checkpoint at xlm-mlm-100-1280 and are newly initialized: ['transformer.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.5245146421449525


In [ ]:
# xlm-roberta-xl
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("facebook/xlm-roberta-xl")
model = AutoModelForMaskedLM.from_pretrained("facebook/xlm-roberta-xl")

get_avg_res_other(tokenizer,model)

C:\Users\Miki\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.938381410070828
